In [1]:
DF_PATH       = "../data/processed/2_cleaned_data.pkl"

ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageWorkedWith',    'DatabaseWorkedWith',    'WebframeWorkedWith',    'MiscTechWorkedWith']



EXPERIMENT_NAME = "stackoverflow_single_baseline_model"
LOG_PATH = "../models"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"


In [ ]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt

In [ ]:
# Read Data 
df = pd.read_pickle(DF_PATH)

In [ ]:
df

DevType                           \
      Academic researcher Data or business analyst   
0                       0                        0   
1                       0                        0   
5                       0                        0   
8                       0                        0   
11                      0                        0   
...                   ...                      ...   
64405                   0                        0   
64416                   0                        1   
64422                   0                        0   
64428                   0                        0   
64447                   0                        0   

                                                                            \
      Data scientist or machine learning specialist Database administrator   
0                                                 0                      0   
1                                                 0                      0   
5                                                 0                      0   
8                                                 0                      0   
11                                                0                      0   
...                                             ...                    ...   
64405                                             0                      0   
64416                                             0                      0   
64422                                             0                      0   
64428                                             0                      0   
64447                                             0                      0   

                                                                   \
      DevOps specialist Developer, QA or test Developer, back-end   
0                     0                     0                   0   
1                     0                     0                   0   
5                     0                     0                   0   
8                     0                     0                   0   
11                    0                     0                   0   
...                 ...                   ...                 ...   
64405                 0                     1                   0   
64416                 0                     0                   0   
64422                 0                     0                   1   
64428                 0                     0                   0   
64447                 0                     0                   1   

                                                     \
      Developer, desktop or enterprise applications   
0                                                 1   
1                                                 0   
5                                                 0   
8                                                 0   
11                                                0   
...                                             ...   
64405                                             0   
64416                                             0   
64422                                             0   
64428                                             0   
64447                                             0   

                                                                        ...  \
      Developer, embedded applications or devices Developer, front-end  ...   
0                                               0                    0  ...   
1                                               0                    0  ...   
5                                               0                    1  ...   
8                                               0                    0  ...   
11                                              0                    1  ...   
...                                           ...                  ...  ...   
64405                                           0     

### Functions

In [ ]:
def calculate_quality(ground_truth, prediction, metric_function, sort_values=False):
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores

### Balance classes

In [ ]:
# Check the total samples of roles
jops_df = df["DevType"].copy()
jops_sum = df["DevType"].sum(axis=0)
jops_sum

Academic researcher                               581
Data or business analyst                          669
Data scientist or machine learning specialist     799
Database administrator                            296
DevOps specialist                                 677
Developer, QA or test                             493
Developer, back-end                              5503
Developer, desktop or enterprise applications    1671
Developer, embedded applications or devices       795
Developer, front-end                             2890
Developer, full-stack                            5578
Developer, game or graphics                       342
Developer, mobile                                1859
Engineer, data                                    483
Scientist                                         292
System administrator                              440
dtype: int64

In [ ]:
# Resample roles
samples_per_class = 500
resampled_roles = []

for role_col in jops_df.columns:
    sub_df = jops_df.loc[jops_df[role_col] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=0) 
    
    resampled_roles.append(sub_df)

In [ ]:
# Construct dfs
jops_df  = pd.concat(resampled_roles)
df = df.loc[jops_df.index]

In [ ]:
jops_df.sum(axis=0)

Academic researcher                               797
Data or business analyst                          767
Data scientist or machine learning specialist     871
Database administrator                            588
DevOps specialist                                 705
Developer, QA or test                             590
Developer, back-end                              1770
Developer, desktop or enterprise applications     892
Developer, embedded applications or devices       666
Developer, front-end                              893
Developer, full-stack                            1431
Developer, game or graphics                       560
Developer, mobile                                 800
Engineer, data                                    631
Scientist                                         642
System administrator                              647
dtype: int64

### splitting 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop("DevType", axis=1), 
                                                    df["DevType"], 
                                                    random_state=0)

C:\Users\pc\AppData\Local\Temp\ipykernel_60196\1098765831.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train, X_test, Y_train, Y_test = train_test_split(df.drop("DevType", axis=1),


In [ ]:
X_train.shape

(6000, 114)

### Train models

In [ ]:
# Initialize client and experiment
client = MlflowClient()
mlflow.set_experiment(EXPERIMENT_NAME)
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

2023/01/01 16:52:14 INFO mlflow.tracking.fluent: Experiment with name 'stackoverflow_single_baseline_model' does not exist. Creating a new experiment.


### Logistic regression

In [ ]:
clf = make_pipeline(StandardScaler(), 
                    MultiOutputClassifier(LogisticRegression()))

clf.fit(X_train, Y_train)
predictions =  pd.DataFrame(clf.predict(X_train), columns=Y_train.columns)

C:\Users\pc\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\pc\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\pc\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
# Evaluate on train set
predictions =  pd.DataFrame(clf.predict(X_train), columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

C:\Users\pc\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
# Evaluate on test set
predictions =  pd.DataFrame(clf.predict(X_test), columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores = test_scores.mean()

C:\Users\pc\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
print(mean_test_scores)
test_scores.sort_values("f1_score")

accuracy_score     91.268750
precision_score    63.117500
recall_score       32.406250
f1_score           41.151875
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
"Developer, QA or test",92.05,63.64,4.32,8.09
Database administrator,93.05,55.00,15.38,24.04
"Developer, desktop or enterprise applications",89.25,52.05,17.43,26.12
Scientist,92.50,49.09,18.12,26.47
System administrator,91.85,55.17,18.93,28.19
"Engineer, data",92.85,67.92,22.22,33.49
Academic researcher,90.65,62.50,24.15,34.84
"Developer, full-stack",84.45,61.73,33.89,43.76
"Developer, embedded applications or devices",92.75,59.79,35.37,44.44
"Developer, back-end",80.80,61.92,36.10,45.61


### Log

In [ ]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [ ]:
# Model
model = {"model_description": "Baseline model: Logistic Regression ",
         "model_details": str(clf),
         "model_object": clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [ ]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [ ]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, run_name=model["model_description"]):
    # Log pickles 
    mlflow.log_artifacts(LOG_PATH)
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 
    